<a href="https://colab.research.google.com/github/ilucas75/BTflow/blob/dev/RD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install

In [1]:
!pip install pdfminer.six
!pip install spacy
!pip install tabula-py
!python -m spacy download fr_core_news_md

     |████████████████████████████████| 5.6MB 2.6MB/s 
     |████████████████████████████████| 13.7MB 320kB/s 
     |████████████████████████████████| 10.4MB 2.7MB/s 
     |████████████████████████████████| 88.6MB 903kB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-cp36-none-any.whl size=90338488 sha256=d5897ecd4ece7a92ee9a50b8ef3aeda69a3986c2bf0892b84036f479eabfe942
  Stored in directory: /tmp/pip-ephem-wheel-cache-kgtzptbz/wheels/c6/18/b6/f628642acc7872a53cf81269dd1c394d96da69564ccfac5425
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


## Upload PDF and Json

In [2]:
from google.colab import files

uploaded = files.upload()


Saving 0070C022_220C0700.json to 0070C022_220C0700.json
Saving 0340C022_220C0430.json to 0340C022_220C0430.json
Saving 0510C022_220C0150.json to 0510C022_220C0150.json
Saving 0540C022_220C0450.json to 0540C022_220C0450.json
Saving 0620C022_220C0260.json to 0620C022_220C0260.json
Saving 0810C022_220C0180.json to 0810C022_220C0180.json
Saving 0820C022_220C0280.json to 0820C022_220C0280.json
Saving 0900C022_220C0090.json to 0900C022_220C0090.json
Saving 0940C022_220C0490.json to 0940C022_220C0490.json
Saving 0960C022_220C0690.json to 0960C022_220C0690.json
Saving 0980C022_220C0890.json to 0980C022_220C0890.json
Saving 1040C022_220C0401.json to 1040C022_220C0401.json
Saving 1080C022_220C0801.json to 1080C022_220C0801.json
Saving 1090C022_220C0901.json to 1090C022_220C0901.json
Saving 1200C022_220C0021.json to 1200C022_220C0021.json
Saving 1270C022_220C0721.json to 1270C022_220C0721.json
Saving 1360C022_220C0631.json to 1360C022_220C0631.json
Saving 1380C022_220C0831.json to 1380C022_220C08

In [0]:
ls

dataset.json  sample_data/


## Table PDF

In [0]:
from tabula import read_pdf
import pandas as pd

def extract_tables(filename):
  dfs = read_pdf(list_pdf[0], pages='all')
  tables_str = []
  for df in dfs:
  if not df.empty:
    table_str = ""
    for c in df.columns:
      if not "Unnamed:" in c:
        table_str += c + " "
    for _, r in df.iterrows():
      for c in df.columns:
        if pd.notna(r[c]):
          table_str += r[c] + " "
    tables_str.append(table_str)
  return dfs, tables_str

Got stderr: Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



## Extract PDF text

In [0]:
import io
 
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [0]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
 
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
 
        text = fake_file_handle.getvalue()
 
    # close open handles
    converter.close()
    fake_file_handle.close()
 
    if text:
        return text

## Load PDF and Json

In [5]:
import json
import os 

data = {}
data_label = {}
data_no_label = {}

for f in uploaded.keys():
  try:
    name, ext = os.path.splitext(f)
    if ext == '.pdf' and name not in pdfs:
      text=extract_text_from_pdf(f)
      if "Déclaration de franchissement de seuils" in text:
        x =  {"file": f, "text": text}
        data[name] = x
        data_no_label[name] = x
    elif ext == '.json':
      with open(f) as fd:
        x = json.load(fd)
        data[name] = x
        if 'entities' in x and len(x['entities']) != 0:
          data_label[name] = x
          if name in data_no_label:
            del data_no_label[name]
        else:
          data_no_label[name] = x
  except Exception as e:
    print("Error : ", f)
    print(e)
data = [v for k, v in sorted(data.items(), key=lambda t: t[0])]
data_label = [v for k, v in sorted(data_label.items(), key=lambda t: t[0])]
data_no_label = [v for k, v in sorted(data_no_label.items(), key=lambda t: t[0])]
print(len(data), "texts with", len(data_label), "labeled and", len(data_no_label), 'to label')

117 texts with 117 labeled and 0 to label


## Labellisation

In [0]:
import IPython
from google.colab import output
import os

class AnnotWidget():
  def __init__(self, to_annot, start=0, model=None):
    self.to_annot = to_annot
    self.cur = start
    self.model = model
    self.colors_labels = [
        ("rgb(255, 0, 0)", "ISSUER", 'A'),
        ("rgb(255, 0, 255)", "FIRM", 'Z'),
        ("rgb(0, 255, 255)", "NUMBER", 'E'),
        ("rgb(160, 160, 255)", "PERCENT", 'R'),
        ("rgb(192, 192, 192)", "TYPE", 'T'),
        ("rgb(255, 255, 0)", "DIRECTION", 'S'),
        ("rgb(0, 255, 0)", "DATE", 'D'),
        ("rgb(255, 180, 180)", "FILIALE", 'F'),
        ("rgb(230, 171, 0)", 'INSTRUMENT', 'G')
      ]
    self.controls = ""
    for c, l, k in self.colors_labels:
      self.controls += '''<button id="key_'''+k+'''" style="background-color: '''+\
        c+''';" onclick='document.execCommand("backColor", false, "'''+c+'''")'>'''+l+" ("+k+")</button>"

  def text_entities_to_html(self):
    if 'entities' not in self.to_annot[self.cur] or len(self.to_annot[self.cur]['entities']) == 0:
      return self.to_annot[self.cur]["text"]
    res = ""
    offset = 0
    text = self.to_annot[self.cur]["text"]
    for ent_s, ent_e, label in self.to_annot[self.cur]['entities']:
      res += text[offset:ent_s]
      res += '''<span style="background-color: '''+ self.get_color(label) + '''">'''
      res += text[ent_s:ent_e]
      res += "</span>"
      offset = ent_e
    res += text[offset:]
    return res

  def run(self):
    display(IPython.display.HTML('''
    <p>'''+str(self.cur) + "/" + str(len(self.to_annot)) + '''</p>
    <fieldset>
    '''+ self.controls +'''
      <button id="key_Q" onclick="document.execCommand('removeFormat', false, null)">remove (Q)</button>
      <button id="key_P" onclick="google.colab.kernel.invokeFunction('notebook.Prediction');">pred (P)</button>
      <button id='prev'>prev</button>
      <button id='next'>next</button>
    </fieldset>
    <div id="ed" contenteditable="true">'''+self.text_entities_to_html()+'''</div>
    <script>
      function change(dir) {
        children = document.querySelector("#ed").childNodes;
        res = [];
        offset = 0;
        for (var i = 0; i < children.length; i++) {
          next_offset = offset + children[i].textContent.length;
          if (children[i].nodeType != Node.TEXT_NODE) {
            rgb = children[i].style.backgroundColor
            res.push([offset, next_offset, rgb])
          }
          offset = next_offset;
        }
        google.colab.kernel.invokeFunction('notebook.SaveEntities', [res, dir]);
        document.querySelector('#ed').innerHTML = next_text;
      };
      document.querySelector('#prev').onclick = () => {
        change(-1);
      };
      document.querySelector('#next').onclick = () => {
        change(1);
      };
      document.querySelector('#ed').onkeydown = (ev) => {
        ev.preventDefault();
      };
      document.querySelector('#ed').onkeypress = (ev) => {
        ev.preventDefault();
      };
      document.querySelector('#ed').onkeyup = (ev) => {
        ev.preventDefault();
        button_id = "#key_" + String.fromCharCode(ev.keyCode);
        button = document.querySelector(button_id)
        if (button) {
          button.click();
        }
      };
    </script>
    '''))
    output.register_callback('notebook.SaveEntities', self.save_entities)
    output.register_callback('notebook.Console', self.console)
    output.register_callback('notebook.Prediction', self.prediction)

  def get_color(self, label):
    for c, l, _ in self.colors_labels:
        if l == label:
          return c

  def get_label(self, color):
    for c, l, _ in self.colors_labels:
        if c == color:
          return l

  def save_entities(self, res, inc=1):
    self.to_annot[self.cur]['entities'] = []
    for ent_s, ent_e, color in res:
      self.to_annot[self.cur]['entities'].append((ent_s, ent_e, self.get_label(color)))
    if len(self.to_annot[self.cur]['entities']) != 0:
      name, _ = os.path.splitext(self.to_annot[self.cur]['file'])
      with open(name+'.json', 'w+') as f:
        json.dump(self.to_annot[self.cur], f)
    print("hello")
    output.clear()
    if (self.cur < len(self.to_annot) and inc == 1) or (self.cur > 0 and inc == -1):
      self.cur += inc
    self.run()

  def console(self, s):
    print(s)

  def prediction(self):
    if (self.model is not None) and (('entities' not in self.to_annot[self.cur]) or (len(self.to_annot[self.cur]['entities']) == 0)):
      doc = self.model(self.to_annot[self.cur]["text"])
      self.to_annot[self.cur]['entities'] = []
      for ent in doc.ents:
        self.to_annot[self.cur]['entities'].append([ent.start_char, ent.end_char, ent.label_])
      output.clear()
      self.run()


In [0]:
w = AnnotWidget(data, start=0, model=None)

In [47]:
w.run()

In [0]:
print(data[0])

{'file': '0070C022_220C0700.pdf', 'text': "220C0700-FR0000034639-FS0185            220C0700 FR0000034639-FS0185 21 février 2020   Déclaration de franchissement de seuils (article L. 233-7 du code de commerce)   ALTRAN TECHNOLOGIES  (Euronext Paris)   Par courrier reçu le 20 février 2020, complété par un courrier reçu le 21 février, la société Bank of America Corporation (Corporation Trust Center, Orange Street, Wilmington, DE 19801, Etats-Unis) a déclaré avoir franchi en hausse, indirectement par l’intermédiaire des sociétés qu’elle contrôle, le 17 février 2020, les seuils de 5% du capital et des droits de vote de la société ALTRAN TECHNOLOGIES et détenir 14 377 895 actions ALTRAN TECHNOLOGIES représentant autant de droits de vote, soit 5,59% du capital et 5,58% des droits de vote de cette société1, répartis comme suit :   Actions % capital Droits de vote % droits de vote Merrill Lynch International 14 101 661 5,49 14 101 661 5,47 BofA Securities Europe SA 276 021 0,11 276 021 0,11 Bof

In [0]:
data = {x['file'] : x for x in data}
data_label = {x['file'] : x for x in data_label}
data_no_label = {x['file'] : x for x in data_no_label}
for x in w.to_annot:
  if 'entities' in x and len(x['entities']) != 0:
    data[x['file']] = x
    data_label[x['file']] = x
    if x['file'] in data_no_label:
      del data_no_label[x['file']]
data = [v for k, v in sorted(data.items(), key=lambda t: t[0])]
data_label = [v for k, v in sorted(data_label.items(), key=lambda t: t[0])]
data_no_label = [v for k, v in sorted(data_no_label.items(), key=lambda t: t[0])]
print(len(data), "texts with", len(data_label), "labeled and", len(data_no_label), 'to label')

117 texts with 117 labeled and 0 to label


## Download Json

In [13]:
!zip dataset.zip *.json

  adding: 0070C022_220C0700.json (deflated 66%)
  adding: 0340C022_220C0430.json (deflated 59%)
  adding: 0510C022_220C0150.json (deflated 51%)
  adding: 0540C022_220C0450.json (deflated 62%)
  adding: 0620C022_220C0260.json (deflated 50%)
  adding: 0810C022_220C0180.json (deflated 69%)
  adding: 0820C022_220C0280.json (deflated 60%)
  adding: 0900C022_220C0090.json (deflated 74%)
  adding: 0940C022_220C0490.json (deflated 56%)
  adding: 0960C022_220C0690.json (deflated 69%)
  adding: 0980C022_220C0890.json (deflated 54%)
  adding: 1040C022_220C0401.json (deflated 61%)
  adding: 1080C022_220C0801.json (deflated 51%)
  adding: 1090C022_220C0901.json (deflated 66%)
  adding: 1200C022_220C0021.json (deflated 58%)
  adding: 1270C022_220C0721.json (deflated 64%)
  adding: 1360C022_220C0631.json (deflated 65%)
  adding: 1380C022_220C0831.json (deflated 55%)
  adding: 1430C022_220C0341.json (deflated 67%)
  adding: 1500C022_220C0051.json (deflated 59%)
  adding: 1550C022_220C0551.json (deflat

In [0]:
from google.colab import files
files.download('dataset.zip') 

## Train NER

In [48]:
split = 0.33
train_data = []
for x in data_label:
  entities = [tuple(ent) for ent in  x["entities"]]
  train_data.append((x['text'], {'entities' : entities}))
labels = ['ISSUER', 'FIRM', 'NUMBER', 'PERCENT', 'TYPE', 'DIRECTION', 'DATE', 'FILIALE', 'INSTRUMENT']
split = int(split * len(train_data))
test_data = train_data[:split]
train_data = train_data[split:]
print(len(test_data), len(train_data))

38 79


In [49]:
import random
import spacy
from spacy.util import minibatch, compounding
import fr_core_news_md

n_iter=40
nlp = fr_core_news_md.load()
ner = nlp.get_pipe("ner")
for label in labels:
  ner.add_label(label)
optimizer = nlp.resume_training()
pipe_exceptions = ["ner"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*other_pipes):  # only train NER
  sizes = compounding(1.0, 4.0, 1.001)
  # batch up the examples using spaCy's minibatch
  for itn in range(n_iter):
    random.shuffle(train_data)
    batches = minibatch(train_data, size=sizes)
    losses = {}
    for batch in batches:
      texts, annotations = zip(*batch)
      nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
    print("Losses", losses)
  score = nlp.evaluate(test_data)

Losses {'ner': 20418.287743776702}
Losses {'ner': 18284.22239235323}
Losses {'ner': 18030.918165029027}
Losses {'ner': 17799.607063446194}
Losses {'ner': 17606.9738421035}
Losses {'ner': 17174.19812640734}
Losses {'ner': 17081.649698018562}
Losses {'ner': 17289.28596050175}
Losses {'ner': 17046.063705135835}
Losses {'ner': 17010.195310413837}
Losses {'ner': 16967.150909990072}
Losses {'ner': 16906.983227610588}
Losses {'ner': 16748.484810269438}
Losses {'ner': 16649.32038795948}
Losses {'ner': 17005.339824080467}
Losses {'ner': 16651.868200540543}
Losses {'ner': 17143.058803899214}
Losses {'ner': 16987.090646848083}
Losses {'ner': 16971.786228060722}
Losses {'ner': 16823.9430270195}
Losses {'ner': 17124.98987966776}
Losses {'ner': 16712.593997212127}
Losses {'ner': 16433.58081331849}
Losses {'ner': 17125.645394325256}
Losses {'ner': 16566.71200978756}
Losses {'ner': 16812.964337415993}
Losses {'ner': 16856.179999113083}
Losses {'ner': 16657.422562658787}
Losses {'ner': 16857.5332868099

In [50]:
score.scores

{'ents_f': 91.04859335038363,
 'ents_p': 89.8989898989899,
 'ents_per_type': {'DATE': {'f': 98.4126984126984, 'p': 96.875, 'r': 100.0},
  'DIRECTION': {'f': 100.0, 'p': 100.0, 'r': 100.0},
  'FILIALE': {'f': 85.71428571428571,
   'p': 85.71428571428571,
   'r': 85.71428571428571},
  'FIRM': {'f': 55.172413793103445,
   'p': 44.44444444444444,
   'r': 72.72727272727273},
  'INSTRUMENT': {'f': 66.66666666666666,
   'p': 66.66666666666666,
   'r': 66.66666666666666},
  'ISSUER': {'f': 91.35802469135803,
   'p': 88.09523809523809,
   'r': 94.87179487179486},
  'NUMBER': {'f': 90.0, 'p': 100.0, 'r': 81.81818181818183},
  'PERCENT': {'f': 97.67441860465117, 'p': 100.0, 'r': 95.45454545454545},
  'TYPE': {'f': 94.62365591397848,
   'p': 95.65217391304348,
   'r': 93.61702127659575}},
 'ents_r': 92.2279792746114,
 'las': 0.0,
 'las_per_type': {'': {'f': 0.0, 'p': 0.0, 'r': 0.0}},
 'tags_acc': 0.0,
 'textcat_score': 0.0,
 'textcats_per_cat': {},
 'token_acc': 100.0,
 'uas': 0.0}

In [54]:
from spacy import displacy
doc = nlp(data[0]['text'])

displacy.render(doc, style='ent', jupyter=True)

In [53]:
for t in test_data:
  doc = nlp(t[0])
  label_set = set([(e_s, e_e, e_t) for e_s, e_e, e_t in t[1]['entities']])
  pred_set = set([(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents])
  #print(label_set)
  print(pred_set - label_set)
  print(label_set - pred_set)


{(279, 290, 'DATE'), (1121, 1127, 'ISSUER'), (590, 609, 'ISSUER')}
{(590, 591, 'ISSUER'), (1121, 1140, 'ISSUER'), (591, 609, 'ISSUER'), (1928, 1947, 'ISSUER'), (2477, 2501, 'INSTRUMENT'), (279, 289, 'DATE')}
{(1109, 1112, 'TYPE'), (511, 516, 'ISSUER'), (1608, 1618, 'INSTRUMENT'), (804, 814, 'INSTRUMENT'), (1113, 1118, 'ISSUER'), (1103, 1108, 'NUMBER'), (589, 595, 'ISSUER')}
{(589, 594, 'ISSUER'), (511, 516, 'FIRM')}
{(171, 190, 'ISSUER'), (552, 562, 'INSTRUMENT')}
{(171, 191, 'ISSUER')}
{(697, 718, 'ISSUER'), (904, 925, 'ISSUER'), (216, 237, 'ISSUER'), (1878, 1892, 'ISSUER'), (1338, 1352, 'ISSUER'), (611, 625, 'ISSUER'), (928, 942, 'ISSUER'), (587, 608, 'ISSUER'), (1314, 1335, 'ISSUER'), (1642, 1663, 'ISSUER'), (297, 311, 'DATE'), (1987, 2008, 'ISSUER'), (1854, 1875, 'ISSUER'), (1666, 1680, 'ISSUER'), (1546, 1567, 'ISSUER'), (240, 254, 'ISSUER')}
{(1987, 2025, 'ISSUER'), (697, 735, 'ISSUER'), (1854, 1892, 'ISSUER'), (587, 625, 'ISSUER'), (216, 254, 'ISSUER'), (1642, 1680, 'ISSUER'), (1

In [127]:
import pandas as pd

def exctract_info(text, entities):
  df = pd.DataFrame.from_records([{'text':text[s:e], 'start':s, 'end':e, 'type':t} for s, e, t in entities])
  ret = {
      'Issuer' : None,
      'Firm' : [],
      'Direction' : None,
      'Date' : None,
      'ThreasholdCapital' : None,
      'TreadholdVote' : None,
      'Capital': None, 
      'Actions' : None,
      'Filiales' : [],
      'Instruments' : [],
  }
  print(df[df['type'] == 'ISSUER']['text'].value_counts().index[0])
  print(df[df['type'] == 'FIRM']['text'].value_counts(sort=False).index.to_list())
  dir_idx = df[df['type'] == 'DIRECTION'].index[0]
  print(df[df['type'] == 'DIRECTION']['text'].iloc[0])
  # CAS MULTI SEUILS
  types = df.copy()
  types[['p_text', 'p_start', 'p_end', 'p_type']] = types[['text', 'start', 'end', 'type']].shift(1)
  types[['p2_text', 'p2_start', 'p2_end', 'p2_type']] = types[['text', 'start', 'end', 'type']].shift(2)
  types = types[types['type'] == 'TYPE']
  def func(r):
    #print(r)
    if r.start - r.p_end < 20: 
      if r.p_type in ['NUMBER', 'PERCENT']:
        return pd.Series([r.p_text, r.p_type])
      elif r.p_type == 'TYPE' and r.p2_type in ['NUMBER', 'PERCENT'] and r.p_start - r.p2_end < 20:
        return pd.Series([r.p2_text, r.p2_type])
    return pd.Series([None, None])

  types[['N', 'N_type']] = types.apply(func, axis = 1)
  print(types[types.text.str.contains('capital') & (types.N_type == 'PERCENT')])
  print(types[types.text.str.contains('vote') & (types.N_type == 'PERCENT')])
  print(types[types.text.str.contains('action') & (types.N_type == 'NUMBER')])
  print(df[(df['type'] == 'DATE') & (df.index > dir_idx)]['text'].iloc[0])
  print(df[df['type'] == 'FILIALE']['text'].value_counts(sort=False).index.to_list())
  print(df[df['type'] == 'INSTRUMENT']['text'].value_counts(sort=False).index.to_list())
  
  dist_max = 15
  linked = []
  quantities = []
  pos_prev = None
  quant_type = None
  prev_type = None
  for ent_start, ent_end, ent_type in entities:
    ent_text = text[ent_start:ent_end]
    #print(ent_text, quantities)
    if ent_type in ['NUMBER', 'PERCENT']:
      if len(quantities) == 0 or (prev_type == ent_type and ent_start - prev_pos < dist_max):
        quantities.append(ent_text)
        quant_type = ent_type
      else:
        quantities = []
    elif ent_type == 'TYPE':
      print(quantities)
      if len(quantities) != 0 and (prev_type in ['NUMBER', 'PERCENT', 'TYPE']) and ent_start - prev_pos < dist_max:
        linked.append((quantities, ent_text, quant_type))
      else:
        quantities = []
    prev_type = ent_type
    prev_pos = ent_end
  print(linked)
      




for d in test_data[:5]:
  exctract_info(d[0], d[1]['entities'])
  print('--------------')

ALTRAN TECHNOLOGIES
['Bank of America Corporation']
en hausse
       text  start  end  type p_text  ...  p2_start  p2_end p2_type      N   N_type
8   capital    546  553  TYPE     5%  ...     509.0   524.0    DATE     5%  PERCENT
16  capital    713  720  TYPE  5,59%  ...     640.0   659.0  ISSUER  5,59%  PERCENT

[2 rows x 14 columns]
              text  start  end  type  ... p2_end  p2_type      N   N_type
9   droits de vote    561  575  TYPE  ...  542.0  PERCENT     5%  PERCENT
18  droits de vote    734  748  TYPE  ...  720.0     TYPE  5,58%  PERCENT

[2 rows x 14 columns]
       text  start   end  type  ...  p2_end  p2_type            N  N_type
13  actions    632   639  TYPE  ...   609.0   ISSUER   14 377 895  NUMBER
25  actions   1563  1570  TYPE  ...  1533.0  FILIALE   13 784 025  NUMBER
30  actions   1920  1927  TYPE  ...  1893.0  FILIALE      274 482  NUMBER
35  actions   2388  2395  TYPE  ...  2362.0  FILIALE       17 749  NUMBER
41  actions   2734  2741  TYPE  ...  2615.0     

In [24]:
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.label_, chunk.root.text)

 220C0426-FR0000121964-FS0115             220C0426 FR0000121964-FS0115 31 janvier 2020    NP 220C0426-FR0000121964-FS0115
de franchissement de seuils NP franchissement
article L. 233 NP article
7 du code de commerce)    KLEPIERRE  (Euronext Paris)    Par courrier NP code
Bankplassen 2 NP Bankplassen
, Sentrum NP Sentrum
les seuils de 5% du capital et des droits de vote de la société KLEPIERRE et détenir 15 072 929 actions KLEPIERRE2 NP seuils
de droits de vote NP droits
soit 4,98% du capital et des droits de vote de cette société3 NP %
Ce franchissement de seuils résulte d’une restitution d’actions KLEPIERRE à titre de collatéral.   NP franchissement
_ NP _
_ NP _
_ NP _
_ NP _
_ NP _
_                                                             NP _
1 La Banque Centrale de Norvège. NP Banque
Dont NP Dont
126 actions détenues à titre de collatéral NP actions
Sur la base d'un capital composé de 302 664 095 actions représentant autant de droits de vote, en application du 2ème alinéa de l